In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly 

from sklearn.tree import DecisionTreeClassifier, plot_tree,  _tree
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import  RandomForestClassifier
from sklearn.impute import SimpleImputer

from joblib import Parallel, delayed

from time import time

import optuna
from optuna.visualization import plot_param_importances, plot_contour,  plot_slice, plot_optimization_history
import os
import optuna
from optuna.visualization import plot_param_importances, plot_contour,  plot_slice, plot_optimization_history
import pickle
from sklearn.model_selection import TimeSeriesSplit
# pd.set_option('display.max_colwidth', None)
# pd.options.display.max_columns = None

In [35]:
# dataset_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/datasets/'
# dataset_file = 'competencia_01.csv'

base_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/'
dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_01_fe.csv'


ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = [202101,202102,202103,202104]
mes_test = 202106

# agregue sus semillas
semillas = [540079, 250829, 314299, 302111, 801007]

data = pd.read_csv(dataset_path + dataset_file)

In [36]:
def ganancia_prob(y_hat, y, prop=1, class_index=1, threshold=0.025):
  @np.vectorize
  def ganancia_row(predicted, actual, threshold=0.025):
    return  (predicted >= threshold) * (ganancia_acierto if actual == 1 else -costo_estimulo)

  return ganancia_row(y_hat[:,class_index], y).sum() / prop



In [37]:
df=data.copy()
df.shape


(981946, 314)

In [38]:
## elimino columnas duplicadas que se me crear al hacer merge de las distintas tablas en sql.. solucionar a futuro
df = df.drop(columns=['foto_mes_1', 'foto_mes_2','foto_mes_3', 'foto_mes_4','foto_mes_5', 'numero_de_cliente_1','numero_de_cliente_2','numero_de_cliente_3','numero_de_cliente_4','numero_de_cliente_5'])

In [39]:
df.shape

(981946, 304)

In [40]:
df.head(6)

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,percent_rank_mcuentas_saldo,percent_rank_mcaja_ahorro,percent_rank_mcomisiones_mantenimiento,percent_rank_cproductos,percent_rank_cdescubierto_preacordado,percent_rank_mtarjeta_visa_consumo,percent_rank_cprestamos_personales,percent_rank_mcuenta_corriente,percent_rank_Master_mlimitecompra,percent_rank_Visa_mlimitecompra
0,780477847,202105,1,0,0,45,163,350.26,-1166.57,187.55,...,0.584956,0.492209,0.014346,0.013704,0.000000,0.000000,0.000000,0.424605,0.896819,0.950265
1,780477847,202106,1,0,0,45,164,404.09,-418.42,313.24,...,0.506944,0.352036,0.014346,0.013704,0.000000,0.000000,0.000000,0.424605,0.896819,0.950265
2,781859013,202102,1,0,0,55,160,-763.30,557.29,1388.06,...,0.076360,0.000069,0.728090,0.237888,0.035647,0.766509,0.960044,0.098946,0.079154,0.378076
3,781859013,202103,1,0,0,55,161,466.36,-590.41,1997.38,...,0.097061,0.000069,0.787311,0.237888,0.035647,0.595944,0.960044,0.106302,0.079154,0.378076
4,781859013,202104,1,0,0,55,162,616.97,-1242.10,1775.46,...,0.094201,0.078536,0.787311,0.237888,0.035647,0.682662,0.948874,0.106745,0.046268,0.378076
5,781859013,202105,1,0,0,55,163,954.38,-957.36,3034.25,...,0.092316,0.072431,0.787311,0.237888,0.035647,0.813182,0.888664,0.133058,0.046268,0.378076


# primero encuentro los parametros del modelo 

In [41]:
X_train = df[df['foto_mes'].isin(mes_train)]
#binarizo
y_train = X_train['clase_ternaria'].map(lambda x: 0 if x == "CONTINUA" else 1)
y_train = y_train.reset_index(drop=True)
X_train = X_train.drop(columns=['clase_ternaria', 'foto_mes'])



X_test = df[df['foto_mes'] == mes_test]
#binarizo
y_test = X_test['clase_ternaria'].map(lambda x: 0 if x == "CONTINUA" else 1)
y_test = y_test.reset_index(drop=True)
X_test = X_test.drop(columns=['clase_ternaria', 'foto_mes'])

## Bagging, entrenar varios modelos por ejemplo rf

In [22]:
# imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
# Xi = imp_mean.fit_transform(X_train)

# def objective(trial):
#     max_depth = trial.suggest_int('max_depth', 5, 15)
#     min_samples_split = trial.suggest_int('min_samples_split', 2, 200)
#     min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 200)
#     max_features = trial.suggest_float('max_features', 0.05, 0.7)

#     model = RandomForestClassifier(
#         n_estimators=100,
#         max_depth=max_depth,
#         min_samples_split=min_samples_split,
#         min_samples_leaf=min_samples_leaf,
#         max_features=max_features,
#         max_samples=0.7,
#         random_state=semillas[0],
#         n_jobs=-1,
#         oob_score=True
#     )

#     model.fit(Xi, y_train)

#     return ganancia_prob(model.oob_decision_function_, y_train)


# storage_name = "sqlite:///" + db_path + "optimization_exp_compe_01_v3.db"
# study_name = "exp_compe_01_v3"

# study = optuna.create_study(
#     direction="maximize",
#     study_name=study_name,
#     storage=storage_name,
#     load_if_exists=True,
# )


[I 2024-09-25 22:49:56,288] A new study created in RDB with name: exp_compe_01_v3


In [23]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
Xi = imp_mean.fit_transform(X_train)

# Validación cruzada en series temporales
tscv = TimeSeriesSplit(n_splits=3)

def objective(trial):
    # Sugerir los hiperparámetros a optimizar
    max_depth = trial.suggest_int('max_depth', 6, 15)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 200)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 200)
    max_features = trial.suggest_float('max_features', 0.05, 0.7)
    
    # Definir el modelo Random Forest
    model = RandomForestClassifier(
        n_estimators=100,  # Puedes ajustar este valor después de optimizar
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        max_samples=0.7,
        random_state=semillas[0],
        n_jobs=-1,
        oob_score=True
    )

    # Realizar la validación cruzada
    ganancia_total = 0
    for train_index, test_index in tscv.split(Xi):
        X_train_split, X_test_split = Xi[train_index], Xi[test_index]
        y_train_split, y_test_split = y_train.iloc[train_index], y_train.iloc[test_index]
        
        # Entrenar el modelo
        model.fit(X_train_split, y_train_split)
        
        # Obtener la ganancia usando la función personalizada 'ganancia_prob'
        ganancia_total += ganancia_prob(model.oob_decision_function_, y_train_split)
    
    # Devolver la ganancia media de la validación cruzada
    return ganancia_total / tscv.get_n_splits()

# Configuración de Optuna
storage_name = "sqlite:///" + db_path + "optimization_exp_compe_01_v4.db"
study_name = "exp_compe_01_v4"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True
)

[I 2024-09-27 17:23:44,488] A new study created in RDB with name: exp_compe_01_v4


In [24]:
study.optimize(objective, n_trials=10)

[I 2024-09-27 17:43:27,069] Trial 0 finished with value: 576569000.0 and parameters: {'max_depth': 14, 'min_samples_split': 185, 'min_samples_leaf': 162, 'max_features': 0.441205191295404}. Best is trial 0 with value: 576569000.0.
[I 2024-09-27 17:59:26,497] Trial 1 finished with value: 560518000.0 and parameters: {'max_depth': 8, 'min_samples_split': 108, 'min_samples_leaf': 175, 'max_features': 0.5560219493867897}. Best is trial 0 with value: 576569000.0.
[I 2024-09-27 18:22:54,127] Trial 2 finished with value: 556581666.6666666 and parameters: {'max_depth': 12, 'min_samples_split': 131, 'min_samples_leaf': 2, 'max_features': 0.5576190752976401}. Best is trial 0 with value: 576569000.0.
[I 2024-09-27 18:35:32,399] Trial 3 finished with value: 565061000.0 and parameters: {'max_depth': 8, 'min_samples_split': 115, 'min_samples_leaf': 76, 'max_features': 0.43765477493992794}. Best is trial 0 with value: 576569000.0.
[I 2024-09-27 18:42:21,863] Trial 4 finished with value: 579098333.3333

KeyboardInterrupt: 

In [25]:
plot_slice(study)

In [26]:
best_trial = study.best_trial
best_model_params = best_trial.params
print("Mejor modelo:", best_model_params)

Mejor modelo: {'max_depth': 14, 'min_samples_split': 159, 'min_samples_leaf': 22, 'max_features': 0.14081411714465888}


In [27]:
model_rf_2 = RandomForestClassifier(
        n_estimators=1000,
        **study.best_params,
        max_samples=0.7,
        random_state=semillas[0],
        n_jobs=-1,
        oob_score=True
    )

In [28]:
model_rf_2.fit(Xi, y_train)

RandomForestClassifier(max_depth=14, max_features=0.14081411714465888,
                       max_samples=0.7, min_samples_leaf=22,
                       min_samples_split=159, n_estimators=1000, n_jobs=-1,
                       oob_score=True, random_state=540079)

In [29]:
# Obtener la importancia de las características
importancias = model_rf_2.feature_importances_

# Crear un dataframe para visualizarlas junto con el nombre de las variables
importancia_df = pd.DataFrame({
    'Feature': Xi.columns,
    'Importance': importancias
}).sort_values(by='Importance', ascending=False)

# Mostrar las variables más importantes
print(importancia_df.head(10))  # Muestra las 10 más importantes

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [30]:
##guardo el modelo
filename = modelos_path + 'exp_compe_01_m2.sav'
pickle.dump(model_rf_2, open(filename, 'wb'))

In [42]:
#levanto el modelo
filename = modelos_path + 'exp_compe_01_m2.sav'
model_rf = pickle.load(open(filename, 'rb'))

In [32]:
X_test.columns

Index(['numero_de_cliente', 'active_quarter', 'cliente_vip', 'internet',
       'cliente_edad', 'cliente_antiguedad', 'mrentabilidad',
       'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen',
       ...
       'percent_rank_mcuentas_saldo', 'percent_rank_mcaja_ahorro',
       'percent_rank_mcomisiones_mantenimiento', 'percent_rank_cproductos',
       'percent_rank_cdescubierto_preacordado',
       'percent_rank_mtarjeta_visa_consumo',
       'percent_rank_cprestamos_personales', 'percent_rank_mcuenta_corriente',
       'percent_rank_Master_mlimitecompra', 'percent_rank_Visa_mlimitecompra'],
      dtype='object', length=302)

In [43]:
# prediccion
predicciones = model_rf_2.predict_proba(X_test)
#guardo el index
indice_baja = model_rf_2.classes_.tolist().index(1)
X_test['prob_baja'] = predicciones[:, indice_baja]
#estimulo solo a los baja
X_test['Predicted'] = (X_test['prob_baja'] > 1/40).astype(int)
# num de cliente y prediccion
resultados = X_test[["numero_de_cliente","Predicted"]].reset_index(drop=True) 
# Guardar 
resultados.to_csv("../../../exp/Competencia_1/K1_003.csv", index=False, sep=",")

c:\Users\Federico\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning:

X has feature names, but RandomForestClassifier was fitted without feature names

